In [1]:
import requests

# URL for the login form
login_url = 'https://api.opencaselist.com/v1/login'

# Login credentials
login_data = {
    'username': 'ashleychen@live.com',
    'password': 's#170506'
}

# Start a session
session = requests.Session()

# Perform the login
response = session.post(login_url, json=login_data)

# Check if login was successful
if response.ok:
    print("Login successful!")
else:
    print("Login failed. Status code:", response.status_code)

# Extract cookies from the requests session
cookies = session.cookies.get_dict()
caselist = 'ndtceda23'


Login successful!


In [2]:
# connect to azure sql db
import pyodbc
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_SQL_CONNECTIONSTRING = os.environ['AZURE_SQL_CONNECTIONSTRING']

# Connect to the Azure SQL database
conn = pyodbc.connect(AZURE_SQL_CONNECTIONSTRING)

print("Connected to Azure SQL database")



Connected to Azure SQL database


In [42]:
# populate teams
# extract data
schools_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools'
schools = session.get(schools_url.format(caselist=caselist)).json()
cursor = conn.cursor()

teams_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools/{school}/teams'
for school in schools:
    # fetch teams for each school
    teams = session.get(teams_url.format(caselist=caselist, school=school['name'])).json()
    
    # insert teams into db
    for team in teams:
        team_id = team['team_id']
        team_school = school['display_name']
        team_code4 = team['name']
        if len(team_code4) != 4:
            continue
        team_code2 = team_code4[0] + team_code4[2]
        team_code2_alt = team_code4[2] + team_code4[0]
        print(team_id, team_school, team_code4, team_code2, team_code2_alt)
        # cursor.execute("insert into teams (team_id, team_school, team_code4, team_code2, team_code2_alt) values (%s, %s, %s, %s, %s)", (team_id, team_school, team_code4, team_code2, team_code2_alt))
        # insert if not exist
        cursor.execute("if not exists (select * from teams where team_id = ?) insert into teams (team_id, team_school, team_code4, team_code2, team_code2_alt) values (?, ?, ?, ?, ?)", (team_id, team_id, team_school, team_code4, team_code2, team_code2_alt))
        
# close cursor
cursor.close()
# commit changes
conn.commit()

89093 Army CoCo CC CC
89095 Army McSh MS SM
89090 Army McTo MT TM
93168 Army OpSh OS SO
89091 Army PePe PP PP
90868 Army SoSt SS SS
89092 Army WiKa WK KW
91309 Baylor CaPe CP PC
88999 Baylor HuDa HD DH
89797 Baylor KeCh KC CK
89184 Baylor KePe KP PK
94841 Baylor LaHu LH HL
90551 Baylor LoPe LP PL
89183 Baylor LoSt LS SL
88780 Binghamton CoSa CS SC
88964 Binghamton KiTr KT TK
88965 Binghamton PiMa PM MP
88966 Binghamton T Ch TC CT
90460 Boston College HuCh HC CH
90461 Boston College McWa MW WM
90462 Boston College NoNo NN NN
94365 Cal State Fullerton AyDe AD DA
90822 Cal State Fullerton AyMa AM MA
88306 Cal State Fullerton DeVa DV VD
86878 Cal State Fullerton LoWi LW WL
88307 Cal State Fullerton MaPe MP PM
89424 Cal State Fullerton Pepe Pp pP
89237 Central Oklahoma GaSm GS SG
92459 Central Oklahoma SmWa SW WS
95259 Cherokee Trail VoCh VC CV
89446 Cornell AhBe AB BA
90426 Cornell AhDy AD DA
89453 Cornell BaCa BC CB
89455 Cornell BaKo BK KB
92934 Cornell CaTh CT TC
89456 Cornell ChMa CM M

In [5]:
import re

tournaments = set(["Northwestern", "JW Patterson", "Binghamton", "UCO", "Gotham", "MAC", "Run for the Roses", "Harvard", "Wake", "Kathryn", "Kansas State", "Kansas", "Mukai", "Rutgers RR", "Dartmouth RR", "Texas", "ADA", "CEDA", "NDT", "Houston"])

bad_to_good = {
    "J.W Patterson": "JW Patterson",
    "J.W. Patterson": "JW Patterson",
    "Kentucky Round Robin": "Run for the Roses",
    "Kentucky RR": "Run for the Roses",
    "Owen L Coon": "Northwestern",
    "Owen L Coon Memorial Debates": "Northwestern",
    "NUSO": "Northwestern",
    "nu": "Northwestern",
    "NU": "Northwestern",
    "Sunflower Swing Part 1": "Kansas",
    "Sunflower Swing pt1": "Kansas",
    "Sunflower Swing Part 1 hosted by KU": "Kansas",
    "Sunflower Swing Part 2": "Kansas State",
    "Sunflower Swing pt2": "Kansas State",
    "Shirley": "Wake",
    "Dartmouth": "Dartmouth RR",
    "Rutgers": "Rutgers RR",
    "ADA": "ADA",
    "CEDA": "CEDA",
    "NDT": "NDT",
    "National Debate Tournament": "NDT",
    "Northwestern": "Northwestern",
}

def clean_tournament_name(name):
    """Remove digits, symbols, and excess whitespace from a tournament name before the first alphabetic letter."""
    # Find the first alphabetic character and split the string at that point
    match = re.search(r'[a-zA-Z]', name)
    if not match:
        return name  # Return the original name if no alphabetic character is found
    
    split_index = match.start()
    prefix = name[:split_index]
    suffix = name[split_index:]
    
    # Clean up the prefix by removing non-alphabetic characters and excess whitespace
    cleaned_prefix = re.sub(r'[^a-zA-Z\s]', '', prefix)
    cleaned_prefix = re.sub(r'\s+', ' ', cleaned_prefix).strip()
    
    # Concatenate the cleaned prefix with the original suffix
    clean_name = cleaned_prefix + suffix
    
    if clean_name in tournaments:
        return clean_name
    
    if clean_name in bad_to_good:
        return bad_to_good[clean_name]
    
    else:
        for tournament in tournaments:
            if tournament.lower() in clean_name.lower():
                return tournament
    
    return clean_name

In [8]:
# populate rounds
rounds_url = 'https://api.opencaselist.com/v1/caselists/{caselist}/schools/{school}/teams/{team}/rounds'
cursor = conn.cursor()
cursor.execute("select team_id, team_code4, team_school, team_code2, team_code2_alt from teams")
bad_to_good = {}
teams = cursor.fetchall()
cursor.close()

# params: round_id, team_id, side [A, N], tournament, round, opponent, opensource
for team in teams:
    team_id = team[0]
    team_code = team[1].split(' ')[-1]
    team_school = team[2]

    response = session.get(rounds_url.format(caselist=caselist, school=team_school, team=team_code))
    rounds = response.json()
    for rnd in rounds:
        side = "aff" if rnd['side'] == 'A' else "neg"
        
        clean_name = clean_tournament_name(rnd['tournament'])
        
        if clean_name in bad_to_good:
            tournament = bad_to_good[clean_name]
        else:
            tournament = input(f"Enter name for {clean_name}: ") # clean_tournament_name(rnd['tournament'])
            bad_to_good[clean_name] = tournament
            
        # Check if the round is open source
        if rnd.get('opensource'):
            if not rnd['opponent']:
                continue
            opponent = rnd['opponent'].split(' ')
            opp_school = ' '.join(opponent[:-1])
            opp_code = opponent[-1]

            # Find opponent team_id
            cursor = conn.cursor()
            cursor.execute(
                "SELECT team_id FROM teams WHERE team_school = ? AND (team_code4 = ? OR team_code2 = ? OR team_code2_alt = ?)",
                (opp_school, opp_code, opp_code, opp_code)
            )
            opp_team_id = cursor.fetchall() # There may be multiple teams with the same code
            # Use the first team_id if there are multiple matches
            opp_team_id = opp_team_id[0] if opp_team_id else None
            # Remove tuple wrapper if there is only one team_id
            opp_team_id = opp_team_id[0] if opp_team_id else None

            opp_side = "aff" if side == "neg" else "neg"
            print("opponent", opp_school, opp_code, opp_team_id)

            # Check if the round already exists
            cursor.execute(
                f"SELECT round_id FROM caselist WHERE tournament_name = ? AND round_name = ? AND {opp_side}_team = ?",
                (tournament, rnd['round'], opp_team_id)
            )
            existing_round = cursor.fetchall()
            existing_round = existing_round[0] if existing_round else None
            # Remove tuple wrapper if there is only one round_id
            existing_round = existing_round[0] if existing_round else None

            # Insert or update the round
            if existing_round:
                cursor.execute(
                    f"UPDATE caselist SET {side}_team = ?, {side}_doc = ? WHERE round_id = ?",
                    (team_id, rnd['opensource'], existing_round)
                )
                print("UPDATE", team_school, team_code, rnd['tournament'], rnd['round'], rnd['side'], rnd['opponent'], rnd['opensource'])
            else:
                cursor.execute(
                    f"INSERT INTO caselist (tournament_name, round_name, {side}_team, {side}_doc) VALUES (?, ?, ?, ?)",
                    (tournament, rnd['round'], team_id, rnd['opensource'])
                )
                print("INSERT", team_school, team_code, rnd['tournament'], rnd['round'], rnd['side'], rnd['opponent'], rnd['opensource'])
            cursor.close()
            conn.commit()
        else:
            continue


# commit changes
conn.commit()

opponent Harvard DS 89043
INSERT Houston YaKh 1---NUSO 1 N Harvard DS ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-1.docx
opponent Michigan HT 89195
UPDATE Houston YaKh 1---NUSO 4 N Michigan HT ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-4.docx
opponent Kansas WW 88836
UPDATE Houston YaKh 1---NUSO 5 N Kansas WW ndtceda23/Houston/YaKh/Houston-YaKh-Neg-1---NUSO-Round-5.docx
opponent Kansas State SS 89217
INSERT Houston YaKh 2---Houston 2 N Kansas State SS ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-2.docx
opponent Texas AD 90362
UPDATE Houston YaKh 2---Houston 3 N Texas AD ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-3.docx
opponent Texas CO 90363
UPDATE Houston YaKh 2---Houston 5 N Texas CO ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Round-5.docx
opponent Trinity WP 90384
UPDATE Houston YaKh 2---Houston Quarters N Trinity WP ndtceda23/Houston/YaKh/Houston-YaKh-Neg-2---Houston-Quarters.docx
opponent Kansas LP 90333
UPDATE Houston Ya

In [10]:
print(bad_to_good)

{'NUSO': 'Northwestern', 'Northwestern': 'Northwestern', 'Houston': 'Houston', 'Harvard': 'Harvard', 'Wake': 'Wake', 'Debate': '', 'Contact Info': '', 'Contact Information': '', 'Long Beach': 'Long Beach', 'ADA': 'ADA', 'Brick City RR': 'Brick City RR', 'Kentucky': 'JW Patterson', 'JW Patterson': 'JW Patterson', 'NDT': 'NDT', 'Texas': 'Texas Open', 'Contact': '', 'Shirley': 'Wake', 'Missouri State Tournament': 'Missouri State', 'Missouri State': 'Missouri State', 'All Tournaments': '', '': '', 'shirley': 'Wake', 'Contact/Disclosure': '', 'Contacts': '', 'CSU': 'Long Beach', 'Kentucky Round Robin': 'Run for the Roses', 'Run for the Roses': 'Run for the Roses', 'Emails': '', 'Districts': '', 'nd Big Tent Online': 'Big Tent', 'Big Tent': 'Big Tent', 'th National Debate Tournament': 'NDT', 'Kathryn': 'Kathryn', 'Owen L Coon Memorial Debates': 'Northwestern', 'Kansas': 'Kansas', 'Kansas State': 'Kansas State', 'Indiana': 'Hoosier', 'Hoosier': 'Hoosier', 'CEDA': 'CEDA', 'Dartmouth': 'Dartmou

In [9]:
bad_names = set()
cursor = conn.cursor()
cursor.execute("select tournament_name from caselist")
for row in cursor.fetchall():
    if row[0] not in tournaments:
        bad_names.add(row[0])

cursor.close()
print(bad_names)

{'BTO', 'ADA Fall', '', 'New School', 'West Point', 'Navy', 'Crowe Warken', 'Sunflower Swing', 'Hoosier Invitational', 'GMU', 'Indiana', 'George-Mason University', 'Legal personhood', 'JV Novice Nats', 'quals', 'Big Tent', 'Ziggy', 'UMN', 'MoState', 'Brick City RR', 'Missouri State', 'Districts', 'UK', 'Long Beach', 'Nukes', 'Phyllis Schatz', 'JV Novice', 'University of Minnesota', 'RRR', 'Val RR', 'Hoosier Invitational HIT', 'Liberty', 'Kentucky', 'Texas Open', 'Anti-trust', 'sunflower spring', 'George Mason Debate Tournament', 'West Point Tournament at USMA', 'legal personhood', 'HIT', 'D3', 'University of Minnesota College Invitational', 'Contact Info', 'Missouri State Tournament', 'Wayne', "The People's Tournament", 'UMinn', 'KU', 'West Point Debate Tournament', 'Minnesota', 'George Mason', 'D4', 'D5', 'Army', 'UKRR', 'USNA', 'University of Kentucky', 'D7', 'uk', 'nd Big Tent Online', 'Missouri State University College Tournament', 'D7777777', 'OLC', 'Westpoint Debate Tournament', 

In [55]:
# if good name in bad name, replace bad name with good name
cursor = conn.cursor()
for bad_name in bad_names:
    good_name = input(f"Enter a good name for {bad_name}: ")
    print(good_name)
    if good_name:
        cursor.execute("update caselist set tournament_name = ? where tournament_name = ?", (good_name, bad_name))
        conn.commit()
        # delete bad name
        # cursor.execute("delete from caselist where tournament_name = ?", (bad_name,))

cursor.close()





Wake

Kansas

Northwestern


Wake

Navy

Phyllis Schatz
Kansas




Kentucky


Val RR


Kansas State


Army
Wake
Long Beach

Val RR
Harvard
D5
Indiana

Kansas


Army
Kansas


Wake
Long Beach

Kansas


Kansas State
Ziggy


Kentucky

Kansas State
Liberty
Kansas

Brick City RR


D3
Kansas State
Kansas

NDT
Brick City RR
Ziggy
Phyllis Schatz
Navy



D5

Navy
Long Beach


MAC
D3
D4
Northwestern
Kansas
Kansas

Liberty




Wayne
D7
Rutgers RR
Long Beach

Val RR
JW Patterson
Ziggy



Long Beach
D7


In [8]:
import csv

# insert youtube links into db
with open('youtube_links.csv', 'r') as file:
    reader = csv.DictReader(file) # tournament_name,round_name,aff_school,aff_code,neg_school,neg_code,yt_link
    cursor = conn.cursor()
    
    for row in reader:
        # Find aff team_id
        cursor.execute(
            "SELECT team_id FROM teams WHERE team_school = ? AND (team_code4 = ? OR team_code2 = ? OR team_code2_alt = ?)",
            (row['aff_school'], row['aff_code'], row['aff_code'], row['aff_code'])
        )
        aff_team_id = cursor.fetchall()
        aff_team_id = aff_team_id[0] if aff_team_id else None
        aff_team_id = aff_team_id[0] if aff_team_id else None
        
        # Find neg team_id
        cursor.execute(
            "SELECT team_id FROM teams WHERE team_school = ? AND (team_code4 = ? OR team_code2 = ? OR team_code2_alt = ?)",
            (row['neg_school'], row['neg_code'], row['neg_code'], row['neg_code'])
        )
        neg_team_id = cursor.fetchall()
        neg_team_id = neg_team_id[0] if neg_team_id else None
        neg_team_id = neg_team_id[0] if neg_team_id else None
        
        # query db to see if row exists
        cursor.execute(
            "SELECT * FROM caselist WHERE tournament_name = ? AND round_name = ? AND aff_team = ? AND neg_team = ?",
            (row['tournament_name'], row['round_name'], aff_team_id, neg_team_id)
        )
        existing_row = cursor.fetchall()
        existing_row = existing_row[0] if existing_row else None
        existing_row = existing_row[0] if existing_row else None
        
        print(existing_row, aff_team_id, neg_team_id, row['yt_link'])
        
        # only bother updating do not care to insert
        cursor.execute(
            "UPDATE caselist SET yt_link = ? WHERE round_id = ?",
            (row['yt_link'], existing_row)
        )
        
    cursor.close()
    conn.commit()

None None 92531 https://www.youtube.com/watch?v=bAbI3vogaYE
None 92531 None https://www.youtube.com/watch?v=32BN-7lZGJ4
None 92531 None https://www.youtube.com/watch?v=Fy_QNapo5Ew
None 92531 88765 https://www.youtube.com/watch?v=0pWgCT5KBDo
None None 92531 https://www.youtube.com/watch?v=I3VnPn8K-hQ
1755 92531 89064 https://www.youtube.com/watch?v=SzfnKl7C7Mg
None 92531 88938 https://www.youtube.com/watch?v=rkyqmv78xAk
None None 92531 https://www.youtube.com/watch?v=8q4kIIOmef0
None 87949 92531 https://www.youtube.com/watch?v=w_PQeTND8qc
None 92531 89108 https://www.youtube.com/watch?v=Pxqx6Eo_asg
None 88915 87582 https://www.youtube.com/watch?v=n8UbH3aOc08
None 88924 87582 https://www.youtube.com/watch?v=J2o94bF6O-U
None 88924 87582 https://www.youtube.com/watch?v=o1DPwiSrtX8
None 87582 88967 https://www.youtube.com/watch?v=KyYUgTVP2KM
None 89186 89108 https://www.youtube.com/watch?v=L4zeQ_xeFgU
None 87874 87582 https://www.youtube.com/watch?v=A7GpP-jNnhI
16897 88792 89186 https://www

In [9]:
# 1755 92531 89064
cursor = conn.cursor()
cursor.execute("select * from caselist where yt_link is not null")
print(cursor.fetchall())
cursor.close()


[(68, 'NDT', '8', 87376, 88938, 'ndtceda23/Wyoming/LaPo/Wyoming-LaPo-Aff-NDT-Round-8.docx', 'ndtceda23/Harvard/ClCh/Harvard-ClCh-Neg-05---NDT-Round-8.docx', 'https://www.youtube.com/watch?v=RO1EO62CvXQ'), (378, 'Dartmouth RR', '2', 88249, 88938, 'ndtceda23/Dartmouth/BaCa/Dartmouth-BaCa-Aff-7---Dartmouth-Round-2.docx', 'ndtceda23/Harvard/ClCh/Harvard-ClCh-Neg-04---Dartmouth-RR-Round-2.docx', 'https://www.youtube.com/watch?v=23eTgUI5N3Q'), (380, 'Dartmouth RR', '4', 89131, 88249, 'ndtceda23/Michigan/DoPh/Michigan-DoPh-Aff-Dartmouth-RR-Round-4.docx', 'ndtceda23/Dartmouth/BaCa/Dartmouth-BaCa-Neg-7---Dartmouth-Round-4.docx', 'https://www.youtube.com/watch?v=UniwtFXoPt4'), (381, 'Dartmouth RR', '5', 88249, 88924, 'ndtceda23/Dartmouth/BaCa/Dartmouth-BaCa-Aff-7---Dartmouth-Round-5.docx', 'ndtceda23/Emory/KeRa/Emory-KeRa-Neg-Dartmouth-RR-Round-5.docx', 'https://www.youtube.com/watch?v=8dCke6hZSwI'), (593, 'NDT', '7', 88541, 93505, 'ndtceda23/Minnesota/DaPa/Minnesota-DaPa-Aff-10--NDT-Round-7.doc

In [5]:
conn.commit()